In [24]:
#nltk kütüphanesini import ediyoruz
import nltk
nltk.download('punkt')



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pcx\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [25]:
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import pos_tag, word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.metrics import confusion_matrix
import re

#veriyi okuyoruz
data = pd.read_table(r'C:\Users\pcx\Desktop\SMSSpamCollection',names=['label','text'])



#ingilizcede stop words olarak geçen ve 'a','when' gibi teoride anlamsız
#kelimeleri stringden atmak için kullanacağımız nesneyi oluşturduk.
stop_words = set(stopwords.words('english')) 

#Stemming işlemi, ek almış bir kelimede bulunabilen ortak ön eklerin ve son eklerin 
#bir listesini dikkate alarak kelimenin başlangıcını veya sonunu atıp kök halini bulmaya çalışır.
porter = nltk.PorterStemmer() 

#ham veriyi düzenlediğimiz fonksiyon
def textDuzenle(text):    
    assert(type(text) == str)  # burada text parametresinin string olup olmadığı kontrol edilir
    
    #alttaki işlemler e_mail adresi,dolar ve euro işareti gibi sembolleri string karşılıklarıyla değiştirmek içindir
    duzenli = re.sub(r'\b[\w\-.]+?@\w+?\.\w{2,4}\b', 'emailaddr', text)
    duzenli = re.sub(r'(http[s]?\S+)|(\w+\.[A-Za-z]{2,4}\S*)', 'httpaddr',
                     duzenli)
    duzenli = re.sub(r'£|\$', 'moneysymb', duzenli)
    duzenli= re.sub(
        r'\b(\+\d{1,2}\s)?\d?[\-(.]?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}\b',
        'phonenumbr', duzenli)
    duzenli = re.sub(r'\d+(\.\d+)?', 'numbr', duzenli)
    duzenli= re.sub(r'[^\w\d\s]', ' ', duzenli)
    duzenli = re.sub(r'\s+', ' ', duzenli)
    duzenli = re.sub(r'^\s+|\s+?$', '', duzenli.lower())
    return ' '.join(
        porter.stem(term) 
        for term in duzenli.split()
        if term not in set(stop_words)
    )

# veri setimizde mail içeriğinin bulunduğu text kısmına yazdığımız fonskiyonu uyguluyoruz.
data['text'] = data['text'].apply(textDuzenle)

# train test split işlemine bağımsız değişken olarak text'i, bağımsız değişken olarak label'ı(ham or spam) gönderiyoruz
# test_size bize verinin yüzde kaçlık kısmında test yapılacağını gösterir. Biz yüzde 10'luk kısmını test için ayırıyoruz
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['label'], test_size = 0.1, random_state = 1)

# bu aşamada x_train olarak elde ettiğimiz öğrenme verisini TfidVectorizer sınıfının fit_transform metoduyla tdf-if
# bilgilerini içeren bir sparse matris haline dönüştürüyoruz.
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)

# burada bir svm(support vector machine) oluşturulur ve bu model elde ettiğimiz eğitim verimizle eğitilir
svm = svm.SVC(C=1000)
svm.fit(X_train, y_train)

# burayı testing setimizi test etmek için kullanıyoruz
X_test = vectorizer.transform(X_test)
y_pred = svm.predict(X_test) 


# Burada confusioun matrisi satır ve sütun isimleriyle beraber bir data frame'e aktardık.
confusionMatrix=pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    index=[['actual', 'actual'], ['spam', 'ham']],
    columns=[['predicted', 'predicted'], ['spam', 'ham']]
)
# Bu dataframe'i yazıdırıyoruz.
confusionMatrix



predicted    
                 spam ham
actual spam       488   1
       ham          2  67

In [26]:
# burada sınıflandırma sonucu oluşan değerleri yorumlamak için kullanılan bazı parametreleri buluyoruz

from sklearn.metrics import classification_report

report = classification_report(y_test, y_pred)  
print(report)

              precision    recall  f1-score   support

         ham       1.00      1.00      1.00       489
        spam       0.99      0.97      0.98        69

    accuracy                           0.99       558
   macro avg       0.99      0.98      0.99       558
weighted avg       0.99      0.99      0.99       558



In [27]:
# burada sınıflandırma verilerinin accuracy oranını buluyoruz

from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test,y_pred)  
print(accuracy)

0.9946236559139785
